In [25]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import partial_trace, Statevector
from qiskit.circuit.library import ZGate, XGate
from qiskit import execute, Aer
from qiskit.compiler import transpile
from qiskit.tools.visualization import plot_histogram
from matplotlib import pyplot as plt
from matplotlib import colors
import numpy as np
import gc
import csv
from pylab import *

In [2]:
def hsgs(N, vector):
    
    circuit = QuantumCircuit(N)
    
    phase = -1
    target_phase = phase
    
    # get the number of ones in each state
    vector_n_ones = get_number_of_ones(N)
    
    all_states_in_binary = get_all_states_in_binary(N)
    
    #print(vector)
    
    #print('::::::: case -|00...00>')
    # case: phase on |0> state, i.e.: -|00...00>
    if vector[0] == target_phase:
        # add Z gate to all qubits, we will have the phase in state |0> plus a global phase equal to -1
        circuit.z(range(N))
        # then the target phase should be inverted (due to the global phase of -1)
        target_phase *= phase
        # update other affected states
        for j in range(len(vector)):
            vector[j] *= phase**vector_n_ones[j]
        #print('updated because -|00...00>:', vector)
        # factor out a -1
        #vector *= phase
        #print('factor out a -1:', vector)
    
    
    #print('::::::: case only one |1>')
    # case: states with only one |1> in any qubit
    indices_with_one_1 = np.where(vector_n_ones == 1)[0]
    for j in indices_with_one_1:
        if vector[j] == target_phase:  # need a phase, then add a Z gate
            #print(all_states_in_binary[j])
            #qubit = np.where(np.flip(all_states_in_binary[j]) == 1)[0]
            qubit = np.where(all_states_in_binary[j] == 1)[0]
            #print('qubit =', qubit)
            circuit.z(N-1-qubit) # inverse order due to Qiskit order: |q1q0>
            # update other affected states
            update_other_phases(vector, all_states_in_binary, qubit, j, phase)
            #print(vector)
            
    
    #print('::::::: case p = 2, ..., N')
    # case: p = 2, ..., N
    for p in range(2, N+1):
        #print('p =',p)
        multiConrolledZ = ZGate().control(num_ctrl_qubits=p-1)
        indices_with_p_1s = np.where(vector_n_ones == p)[0]
        for j in indices_with_p_1s:
            if vector[j] == target_phase:  # need a phase, then add a C^pZ gate
                #print(all_states_in_binary[j])
                qubits = np.where(all_states_in_binary[j] == 1)[0]
                #print('qubits =', qubits)
                # inverse order due to Qiskit order: |q1q0>
                qubits_inverted = np.where(np.flip(all_states_in_binary[j]) == 1)[0]
                circuit.append(multiConrolledZ, list(qubits_inverted))
                # update other affected states
                update_other_phases(vector, all_states_in_binary, qubits, j, phase)
                #print(vector)
    
    return circuit

In [3]:
def get_binary_vector(N, k):
    convert_phases = lambda j: (-1)**j
    bitstring_array = np.array(list(format(k, '0'+str(2**N)+'b')), dtype=int)
    vfunc = np.vectorize(convert_phases)
    return vfunc(bitstring_array)

def get_number_of_ones(N):
    return np.array([n.bit_count() for n in range(2**N)])

def get_all_states_in_binary(N):
    return np.array([np.array(list(format(n, '0'+str(N)+'b')), dtype=int) for n in range(2**N)])

def update_other_phases(vector, all_states_in_binary, affected_qubits, j, phase):
    #print('affected_qubits =', affected_qubits)
    for i in range(j, len(vector)):
        #print('all_states_in_binary[',i,'] =', all_states_in_binary[i])
        #print('np.where(all_states_in_binary[',i,']==1)[0] =', np.where(all_states_in_binary[i]==1)[0])
        if all([elem in np.where(all_states_in_binary[i] == 1)[0] for elem in affected_qubits]):
            #print('entro a cambiar el vector')
            vector[i] *= phase
            #print(vector)

In [4]:
def U_i(N, ki, circuit):
    
    # create equal superposition
    circuit.h(range(N))
    
    # get vector i given the value ki
    vector_i = get_binary_vector(N, ki)
    
    # execute hsgs
    circuit.compose(hsgs(N, vector_i), inplace=True)

def U_w(N, kw, circuit):
    
    # get vector w given the value kw
    vector_w = get_binary_vector(N, kw)
    
    # execute hsgs
    circuit.compose(hsgs(N, vector_w), inplace=True)
    
    # apply Hadamards
    circuit.h(range(N))
    
    # apply NOTs
    circuit.x(range(N))

In [5]:
def perceptron_optimized(N, ki, kw, draw=False):
    
    # create circuit, N qubits + ancilla
    circuit = QuantumCircuit(N + 1, 1)
    
    # apply U_i
    U_i(N, ki, circuit)
    
    # apply U_w
    U_w(N, kw, circuit)
    
    # apply C^N X
    circuit.mcx(control_qubits=[i for i in range(N)], target_qubit=N)
    
    # measure the ancilla qubit
    circuit.measure(N, 0)
    
    # draw circuit
    if draw == True:
        display(circuit.draw('mpl'))
    
    return circuit

In [8]:
#!pip install qiskit-ibm-runtime

In [9]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator

In [15]:
#QiskitRuntimeService.save_account(channel='ibm_quantum', token='my_token', overwrite=True)

In [16]:
service = QiskitRuntimeService(channel='ibm_quantum')

In [17]:
service.backends()

[<IBMBackend('ibmq_guadalupe')>,
 <IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('ibm_nairobi')>,
 <IBMBackend('ibm_oslo')>,
 <IBMBackend('ibmq_lima')>,
 <IBMBackend('ibmq_belem')>,
 <IBMBackend('ibmq_quito')>,
 <IBMBackend('simulator_statevector')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibmq_manila')>]

In [18]:
backend = service.backends(simulator=True)[0]
print(backend)

<IBMBackend('ibmq_qasm_simulator')>


In [19]:
backend_belem = service.backends(name='ibmq_belem')[0]
backend_belem

<IBMBackend('ibmq_belem')>

In [20]:
from qiskit.providers.fake_provider import FakeBelemV2
from qiskit_aer.noise import NoiseModel

# Import FakeBackend
fake_backend = FakeBelemV2()
noise_model = NoiseModel.from_backend(fake_backend)

# Set options to include noise_model
options = Options(simulator={
    "noise_model": noise_model
}, resilience_level=0)

In [23]:
%%time

# Calculate for every value for ki and kw

N = 3

jobs_executed = {}

for kw in range(2**(2**N)):
#for kw in range(234, 2**(2**N)):
    
    print("\rworking with kw = {}/{}".format(kw, 2**(2**N)), end="")
    
    circuits_sim = []
    
    for ki in range(2**(2**N)):
        perceptron_circuit = perceptron_optimized(N, ki, kw)
        circuits_sim.append(transpile(perceptron_circuit, backend_belem))
    
    with Session(service=service, backend=backend):
        sampler = Sampler(options=options)
        job = sampler.run(circuits=circuits_sim)
        result = job.result()
        
    jobs_executed[kw] = job.job_id()
    
    with open("res_optimized/results_fake_belem_optimized_Ui_Uw_N3_kw_{}.txt".format(kw), "w") as file:
        file.write(str(result.quasi_dists))
    
    del circuits_sim
    del sampler
    del job
    del result
    gc.collect()
    

with open("res_optimized/results_fake_belem_optimized_Ui_Uw_N3_jobs.txt", "w") as file:
    file.write(str(jobs_executed))

working with kw = 255/256CPU times: total: 12min 18s
Wall time: 30min 39s


In [26]:
%%time
# load results from generated files and save them in matrix form in a new file

N = 3

with open("res_optimized/results_fake_belem_optimized_Ui_Uw_N3_all.txt", "w", newline="") as f:
    
    writer = csv.writer(f, delimiter=' ')
    
    for kw in range(2**(2**N)):

        print("\rreading for kw = {}/{}".format(kw, 2**(2**N)), end="")

        with open("res_optimized/results_fake_belem_optimized_Ui_Uw_N3_kw_{}.txt".format(kw), "r") as file:
            results_for_kw = eval(file.readline())

        results_for_state1 = []
        for ki in range(2**(2**N)):
            results_for_state1.append(results_for_kw[ki][1])
        
        writer.writerow(results_for_state1)

        del results_for_kw
        del results_for_state1
        gc.collect()
    

reading for kw = 255/256CPU times: total: 30.7 s
Wall time: 37.6 s


In [30]:
%%time

# calculate average discrepancy

N = 3

D = 0

for kw in range(2**(2**N)):
    
    print("\rcalculating for kw = {}/{}".format(kw, 2**(2**N)), end="")

    with open("res_optimized/results_fake_belem_optimized_Ui_Uw_N3_kw_{}.txt".format(kw), "r") as file:
        results_for_kw_opt = eval(file.readline())
    
    results_for_state1_opt = []
    for ki in range(2**(2**N)):
        results_for_state1_opt.append(results_for_kw_opt[ki][1])
    
    with open("res_exact/results_fake_belem_exact_Ui_Uw_N3_kw_{}.txt".format(kw), "r") as file:
        results_for_kw_exact = eval(file.readline())
    
    results_for_state1_exact = []
    for ki in range(2**(2**N)):
        results_for_state1_exact.append(results_for_kw_exact[ki][1])
    
    D += np.sum(np.absolute(np.subtract(results_for_state1_opt, results_for_state1_exact))) / (2**(2**(N+1)))
    
print()

D

calculating for kw = 255/256
CPU times: total: 2.23 s
Wall time: 4.26 s


0.10565004348754882